In [1]:
%load_ext autoreload
%autoreload 2

In [7]:
import pickle
from rich import print as printr


with open("/opt/files/maio2022/SAT/NSNet/SATSolving/3-sat/valid/marginals.pkl", "rb") as f:
    marginals = pickle.load(f)

printr(marginals[:4])

[
    array([0.        , 0.        , 0.33333333, 0.        , 0.5       ,
       1.        , 1.        , 0.        , 0.        , 1.        ,
       0.        , 1.        , 0.        , 1.        , 1.        ,
       1.        , 0.33333333, 1.        , 1.        , 1.        ,
       1.        ]),
    array([0. , 0.5, 1. , 1. , 0.5, 1. , 0. , 0.5, 1. , 0.5, 0. , 1. , 0. ,
       0. , 0. ]),
    array([0., 1., 1., 1., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0.]),
    array([0.        , 0.        , 1.        , 0.        , 0.66666667,
       0.        , 0.        , 1.        , 0.66666667, 0.        ,
       1.        , 0.        , 0.        , 0.66666667, 1.        ,
       0.66666667, 0.        , 1.        , 0.        , 1.        ,
       1.        ])
]

### Test 3-SAT generator

In [16]:
import os
import argparse
import random
import networkx as nx

from concurrent.futures.process import ProcessPoolExecutor
from pysat.solvers import Cadical153
from cnfgen import RandomKCNF
from utils.utils import write_dimacs_to, VIG


class Generator:
    def __init__(self, opts):
        self.opts = opts
        os.makedirs(self.opts.out_dir, exist_ok=True)

    def run(self, t):
        if t % self.opts.print_interval == 0:
            print('Generating instance %d.' % t)

        while True:
            n_vars = random.randint(self.opts.min_n, self.opts.max_n)
            n_clauses = int(4.258 * n_vars + 58.26 * pow(n_vars, -2 / 3.))

            cnf = RandomKCNF(3, n_vars, n_clauses)
            clauses = list(cnf.clauses())
            # clauses = [list(cnf._compress_clause(clause)) for clause in clauses]
            vig = VIG(n_vars, clauses)
            if not nx.is_connected(vig):
                continue

            solver = Cadical153(bootstrap_with=clauses)

            if solver.solve():
                write_dimacs_to(n_vars, clauses, os.path.abspath(os.path.join(self.opts.out_dir, '%.5d.cnf' % (t))))
                break

In [17]:
from utils.options import ArgOpts              # custom-made

opts = ArgOpts(
    # --min_n 10 --max_n 40
    min_n=10,
    max_n=40,
    out_dir="/opt/files/maio2022/SAT/NSNet/SATSolving/3-sat/train",
    print_interval=5,
)

In [18]:
generator = Generator(opts)

In [19]:
generator.run(10)

Generating instance 10.


In [ ]:
from cnfgen import RandomKCNF
from utils.utils import write_dimacs_to, VIG


class Generator:
    def __init__(self, opts):
        self.opts = opts
        os.makedirs(self.opts.out_dir, exist_ok=True)

    def run(self, t):
        if t % self.opts.print_interval == 0:
            print('Generating instance %d.' % t)

        while True:
            n_vars = random.randint(self.opts.min_n, self.opts.max_n)
            n_clauses = int(4.258 * n_vars + 58.26 * pow(n_vars, -2 / 3.))

            cnf = RandomKCNF(3, n_vars, n_clauses)
            clauses = list(cnf.clauses())
            clauses = [list(cnf._compress_clause(clause)) for clause in clauses]

### Test data loader

In [1]:
from rich import print as printr

from utils.dataloader import get_dataloader
from utils.options import ArgOpts              # custom-made


opts = ArgOpts.get_default()
printr(opts)

train_loader = get_dataloader(opts.train_dir, opts=opts, mode='train', data_size=opts.train_size)

len(train_loader)

ArgOpts(train_dir='/opt/files/maio2022/SAT/NSNet/SATSolving/SATLIB', batch_size=32, num_workers=4, model='NSNet', 
task='sat-solving', loss='marginal', train_size=None)

1447

In [2]:
data = next(iter(train_loader))

In [3]:
data.c_size

tensor([ 423,  403,  449,  435,  430,  449,  435,  441,  403,  423,  418,   91,
         441, 3100,  441,  423,  435,  418,  403,  411,  403,  423,  449,  449,
         435,  441,  429,  302,  411,  435,  403,  418])

In [8]:
data[0]

BPG(l_size=[1], c_size=[1], sign_l_edge_index=[2538], c2l_msg_repeat_index=[31640], c2l_msg_scatter_index=[31640], l2c_msg_aggr_repeat_index=[17766], l2c_msg_aggr_scatter_index=[17766], l2c_msg_scatter_index=[8883], c_batch=[423], l_edge_index=[1269], c_edge_index=[1269])

### Torch scatter

In [16]:
import torch

src = torch.Tensor([[2, 0, 1, 4, 3], [0, 2, 1, 3, 4]])
index = torch.tensor([[4, 5, 4, 2, 3], [0, 0, 2, 2, 1]])
out = src.new_ones((2, 6))

Expected:

    out == tensor([[0., 0., 4., 3., 2., 0.],
                   [2., 4., 3., 0., 0., 0.]])
    argmax == tensor([[-1, -1,  3,  4,  0,  1],
                      [ 1,  4,  3, -1, -1, -1]])

In [17]:
out.scatter_reduce_(-1, index, src, reduce="amax", include_self=False)

tensor([[1., 1., 4., 3., 2., 0.],
        [2., 4., 3., 1., 1., 1.]])

In [18]:
out

tensor([[1., 1., 4., 3., 2., 0.],
        [2., 4., 3., 1., 1., 1.]])

In [19]:
# scatter add
src = torch.Tensor([[2, 0, 1, 4, 3], [0, 2, 1, 3, 4]])
index = torch.tensor([[4, 5, 4, 2, 3], [0, 0, 2, 2, 1]])
out = src.new_zeros((2, 6))

Should get:

    tensor([[0., 0., 4., 3., 3., 0.],
            [2., 4., 4., 0., 0., 0.]])

In [20]:
out.scatter_add_(-1, index, src)

tensor([[0., 0., 4., 3., 3., 0.],
        [2., 4., 4., 0., 0., 0.]])

### Test device -> ["mps" (Metal Programming Framework) for MacBook](https://github.com/karpathy/nanoGPT/issues/28)
- https://pytorch.org/docs/stable/notes/mps.html

In [1]:
import torch
torch.backends.mps.is_available()

True

### Show available Cuda memory

In [3]:
import torch

# Select GPU device
device = torch.device('cuda:0')

# Get GPU memory info
print(torch.cuda.get_device_name(device))
print('Memory Usage:')
print('Allocated:', round(torch.cuda.memory_allocated(device)/1024**3,1), 'GB')
print('Cached:   ', round(torch.cuda.memory_reserved(device)/1024**3,1), 'GB')
print('Available:', round(torch.cuda.get_device_properties(device).total_memory/1024**3,1), 'GB')

NVIDIA GeForce RTX 3080 Laptop GPU
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB
Available: 16.0 GB


### Test train

In [1]:
from train_model import main as train_run

In [4]:
import torch
from rich import print as printr
from utils.options import ArgOpts              # custom-made
import os

# opts = ArgOpts(
#     train_dir="/opt/files/maio2022/SAT/NSNet/SATSolving/3-sat/train",
#     epochs=1,
# )

opts = ArgOpts.get_default_train()
opts.epochs = 2

log_dir = os.path.join('runs', opts.exp_id)
checkpoint_dir = os.path.join(log_dir, 'checkpoints')
opts.set("restore", os.path.join(checkpoint_dir, "model_3.pt"))

printr(opts)

torch.autograd.set_detect_anomaly(True)

ArgOpts(train_dir='/opt/files/maio2022/SAT/NSNet/SATSolving/sr/train', restore='runs/NSNet/checkpoints/model_2.pt',
valid_dir='/opt/files/maio2022/SAT/NSNet/SATSolving/sr/valid', task='sat-solving', model='NSNet', num_workers=4, 
batch_size=64, epochs=1, exp_id='NSNet', loss='marginal', train_size=None)

In [5]:
"""
Training LR: 0.000100, Training loss: 1.405417
Training accuracy: 0.010567
Validating...
Validating loss: 2.112922
Validating accuracy: 0.000000
---
100%|##########| 469/469 [03:00<00:00,  2.59it/s]
Training LR: 0.000100, Training loss: 2.121072
Training accuracy: 0.000000
Validating...
Validating loss: 2.019921
Validating accuracy: 0.000000

"""

train_run(opts=opts)

### Manual train loop - detect where NaNs come from

In [4]:

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import os
import sys
import argparse
import numpy as np
import random
import math

from tqdm import tqdm

from torch.optim.lr_scheduler import ReduceLROnPlateau, StepLR

# pytorch-scatter ported to pytorch main
# https://github.com/rusty1s/pytorch_scatter/issues/241#issuecomment-1336116049
# from torch_scatter import scatter_sum
# from torch.scatter_reduce import scatter_sum
from utils.scatter import scatter_sum

from utils.options import add_model_options
from utils.logger import Logger
from utils.dataloader import get_dataloader
from utils.utils import safe_log
from models.nsnet import NSNet
from models.neurosat import NeuroSAT

from utils.options import ArgOpts

opts = None
# def main(opts: ArgOpts = None):


torch.cuda.empty_cache()            # CLEAR CUDA MEMORY??

torch.autograd.set_detect_anomaly(True)


# opts = opts or ArgOpts()            # empty -> uses defaults passed below
opts = ArgOpts(
    train_dir="/opt/files/maio2022/SAT/NSNet/SATSolving/3-sat/train",
    epochs=1,
)

opts = ArgOpts.get_default_train()
opts.epochs = 1


parser = argparse.ArgumentParser()
# parser.add_argument('task', type=str, choices=['model-counting', 'sat-solving'], help='Experiment task')
# parser.add_argument('exp_id', type=str, help='Experiment id')
# parser.add_argument('train_dir', type=str, help='Directory with training data')
parser.add_argument(
    '--task', type=str, choices=['model-counting', 'sat-solving'], help='Experiment task',
    default=opts.get("task", "sat-solving"))
parser.add_argument(
    '--exp_id', type=str, help='Experiment id',
    default=opts.get("exp_id", "NSNet"))
parser.add_argument(
    '--train_dir', type=str, help='Directory with training data',
    default=opts.get("train_dir", "/opt/files/maio2022/SAT/NSNet/SATSolving/sr/train"))

parser.add_argument('--train_size', type=int, default=opts.get("train_size", None), help='Number of training data')
parser.add_argument('--valid_dir', type=str, default=opts.get("valid_dir", None), help='Directory with validating data')
parser.add_argument('--loss', type=str, choices=['assignment', 'marginal'], default=opts.get("loss", 'marginal'), help='Loss type for SAT solving')
parser.add_argument('--restore', type=str, default=opts.get("restore", None), help='Continue training from a checkpoint')
parser.add_argument('--save_model_epochs', type=int, default=opts.get("save_model_epochs", 1), help='Number of epochs between model savings')
parser.add_argument('--num_workers', type=int, default=opts.get("num_workers", 8), help='Number of workers for data loading')
parser.add_argument('--batch_size', type=int, default=opts.get("batch_size", 128), help='Batch size')
parser.add_argument('--epochs', type=int, default=opts.get("epochs", 200), help='Number of epochs during training')
parser.add_argument('--lr', type=float, default=opts.get("lr", 1e-4), help='Learning rate')
parser.add_argument('--weight_dacay', type=float, default=opts.get("weight_decay", 1e-10), help='L2 regularization weight')
parser.add_argument('--scheduler', type=str, default=opts.get("scheduler", None), help='Scheduler')
parser.add_argument('--lr_step_size', type=int, default=opts.get("lr_step_size", 20), help='Learning rate step size')
parser.add_argument('--lr_factor', type=float, default=opts.get("lr_factor", 0.5), help='Learning rate factor')
parser.add_argument('--lr_patience', type=int, default=opts.get("lr_patience", 20), help='Learning rate patience')
parser.add_argument('--clip_norm', type=float, default=opts.get("clip_norm", 0.65), help='Clipping norm')
parser.add_argument('--seed', type=int, default=opts.get("seed", 0), help='Random seed')

add_model_options(parser, opts=opts)

# opts = parser.parse_args()
# Parse arguments when using Jupyter Notebook: https://stackoverflow.com/a/72670647/11750716
opts, _unknown = parser.parse_known_args()

torch.manual_seed(opts.seed)
torch.cuda.manual_seed(opts.seed)
torch.cuda.manual_seed_all(opts.seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(opts.seed)
random.seed(opts.seed)

# opts.log_dir = os.path.join('runs', opts.exp_id)
# opts.checkpoint_dir = os.path.join(opts.log_dir, 'checkpoints')

# os.makedirs(opts.log_dir, exist_ok=True)
# os.makedirs(opts.checkpoint_dir, exist_ok=True)

# opts.log = os.path.join(opts.log_dir, 'log.txt')
# sys.stdout = Logger(opts.log, sys.stdout)
# sys.stderr = Logger(opts.log, sys.stderr)

opts.device = "cpu"
if torch.cuda.is_available():
    opts.device = "cuda"
elif torch.backends.mps.is_available():
    # Check that MPS is available (MacBook with M1 / M2 chip)
    # https://pytorch.org/docs/stable/notes/mps.html
    opts.device = "mps"

# opts.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(opts)

models = {
    'NSNet': NSNet,
    'NeuroSAT': NeuroSAT,
}


with torch.no_grad():

    model = models[opts.model](opts)
    model.to(opts.device)

    optimizer = optim.Adam(model.parameters(), lr=opts.lr, weight_decay=opts.weight_dacay)
    train_loader = get_dataloader(opts.train_dir, opts, 'train', opts.train_size)

    best_loss = float('inf')

    start_epoch = 0

    for epoch in range(start_epoch, start_epoch + opts.epochs):
        print('EPOCH #%d' % epoch)
        print('Training...')
        train_loss = 0
        train_tot = 0
        train_rmse = 0
        train_cnt = 0

        for data in tqdm(train_loader):

            data = data.to(opts.device)
            batch_size = data.c_size.shape[0]

            data_copy = data.clone()

            v_prob = model(data)

            if v_prob.isnan().sum() > 0:
                raise ValueError("WE HAVE NANS IN PRED")

            c_size = data.c_size.sum().item()
            c_batch = data.c_batch
            l_edge_index = data.l_edge_index
            c_edge_index = data.c_edge_index

            preds = v_prob
            labels = data.y
            labels = torch.stack([labels, 1-labels], dim=1)
            loss = F.kl_div(safe_log(preds), labels)

            v_assign = (v_prob > 0.5).float()
            l_assign = v_assign.reshape(-1)
            c_sat = torch.clamp(scatter_sum(l_assign[l_edge_index], c_edge_index, dim=0, dim_size=c_size), max=1)
            sat_batch = (scatter_sum(c_sat, c_batch, dim=0, dim_size=batch_size) == data.c_size).float()
            train_cnt += sat_batch.sum().item()

            # train_loss += loss.item() * batch_size
            # train_tot += batch_size
            # loss.backward()
            # torch.nn.utils.clip_grad_norm_(model.parameters(), opts.clip_norm)
            # optimizer.step()

        train_loss /= train_tot
        print('Training LR: %f, Training loss: %f' % (optimizer.param_groups[0]['lr'], train_loss))

        train_acc = train_cnt / train_tot
        print('Training accuracy: %f' % train_acc)



Namespace(task='sat-solving', exp_id='NSNet', train_dir='/opt/files/maio2022/SAT/NSNet/SATSolving/sr/train', train_size=None, valid_dir='/opt/files/maio2022/SAT/NSNet/SATSolving/sr/valid', loss='marginal', restore=None, save_model_epochs=1, num_workers=4, batch_size=64, epochs=1, lr=0.0001, weight_dacay=1e-10, scheduler=None, lr_step_size=20, lr_factor=0.5, lr_patience=20, clip_norm=0.65, seed=0, model='NSNet', dim=64, n_rounds=10, n_mlp_layers=3, activation='relu', device='cuda')
EPOCH #0
Training...


  0%|                                                                                                               | 0/469 [00:00<?, ?it/s]/opt/miniconda3/envs/tf2/lib/python3.10/site-packages/torch/nn/functional.py:2916: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  warnings.warn(
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 469/469 [00:52<00:00,  8.95it/s]


ZeroDivisionError: division by zero